<a href="https://colab.research.google.com/github/alexsalman/DeepLearningAssignments/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

# Assignment 1

**DUE: Sunday October 10 at 11:59pm**

Turn in the assignment via Canvas.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/).

Before you turn this problem in, make sure everything runs as expected. First, restart the kernel (in the menubar, select Runtime→→Restart runtime) and then run all cells (in the menubar, select Runtime→→Run All).

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE", as well as your name below:

In [2]:
NAME = ""
STUDENT_ID = ""

## Covid-19 Case Prediction

In this Assignment you will learn:


*   Solving a Regression problem using Deep Neural Networks
*   Basic Deep Learning training tips such as hyperparameter tuning and optimization
*   Get familiar with TensorFlow

**Task:** Given the survey results in the past 2 days, in a specific State, predict the percentage of newly tested positive cases for the 3rd day (in percentage).

Data includes the following information (each row includes features for all three days): 

*   States - Through 40 one-hot encoded State information
*   Covid-like illnesses (cli, etc.) (4 features - in percentage)
*   Behavior indicators (wearing masks, etc.) (8 features - in percentage)
*   Mental health indicators (depressed, etc.) (5 features - in percentage)

**Evaluation Metric**: Root Mean Squared Error





In [3]:
import numpy as np
import pandas as pd
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
downloaded = drive.CreateFile({'id':"1wrk4vvTxnp1_3Ftx7qp9uo4IUNBWPAA8"}) 
downloaded.GetContentFile('covid.csv')

# Finally we can actually read in the data.
data = pd.read_csv('covid.csv')

### Part 1) Data preparation

In this part you will complete the following steps to prepare your data for training:

*   Understand the data,
*   Separate label from features,
*   Normalize features, and
*   Split the data into train and test (and later validation)


Understand the *data*

In [139]:
data.columns

Index(['id', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'FL', 'GA', 'ID', 'IL',
       'IN', 'IA', 'KS', 'KY', 'LA', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'NE',
       'NV', 'NJ', 'NM', 'NY', 'NC', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'TX',
       'UT', 'VA', 'WA', 'WV', 'WI', 'cli', 'ili', 'hh_cmnty_cli',
       'nohh_cmnty_cli', 'wearing_mask', 'travel_outside_state',
       'work_outside_home', 'shop', 'restaurant', 'spent_time', 'large_event',
       'public_transit', 'anxious', 'depressed', 'felt_isolated',
       'worried_become_ill', 'worried_finances', 'tested_positive', 'cli.1',
       'ili.1', 'hh_cmnty_cli.1', 'nohh_cmnty_cli.1', 'wearing_mask.1',
       'travel_outside_state.1', 'work_outside_home.1', 'shop.1',
       'restaurant.1', 'spent_time.1', 'large_event.1', 'public_transit.1',
       'anxious.1', 'depressed.1', 'felt_isolated.1', 'worried_become_ill.1',
       'worried_finances.1', 'tested_positive.1', 'cli.2', 'ili.2',
       'hh_cmnty_cli.2', 'nohh_cmnty_cli.2', 'wea

In [127]:
## YOUR CODE HERE ##
print('40 States Information')
pd.set_option("display.max_columns", len(data.columns))
data.iloc[-5:,1:41]
#train=data.sample(frac=0.9,random_state=200) #random state is a seed value
#test=data.drop(train.index)

40 States Information


,AL,AK,AZ,AR,CA,CO,CT,FL,GA,ID,IL,IN,IA,KS,KY,LA,MD,MA,MI,MN,MS,MO,NE,NV,NJ,NM,NY,NC,OH,OK,OR,PA,RI,SC,TX,UT,VA,WA,WV,WI
2695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [131]:
print('4 Features Covid-like illnesses')
pd.set_option("display.max_columns", len(data.columns))
data.iloc[-5:,41:45]

4 Features Covid-like illnesses


,cli,ili,hh_cmnty_cli,nohh_cmnty_cli
2695,0.655823,0.659976,25.265366,20.468897
2696,0.598352,0.602552,25.299465,20.756444
2697,0.586713,0.597559,25.271178,20.770195
2698,0.576435,0.595312,24.607461,20.176201
2699,0.562426,0.572969,24.020275,19.654514


In [136]:
print('8 Features Behavior indicators')
pd.set_option("display.max_columns", len(data.columns))
data.iloc[-5:,45:54]

8 Features Behavior indicators


,wearing_mask,travel_outside_state,work_outside_home,shop,restaurant,spent_time,large_event,public_transit,anxious
2695,91.011756,6.801897,32.727184,50.265694,15.188547,31.597793,8.013637,1.768811,14.699027
2696,90.682057,7.152368,33.638563,50.050349,15.462823,31.656358,8.239559,1.789015,14.808636
2697,90.866100,6.857209,33.959012,50.024971,15.090116,30.839219,7.849525,1.760094,14.617563
2698,90.846126,6.851475,33.932384,49.885129,14.779264,30.617100,7.754800,1.780730,14.513419
2699,90.928655,6.642911,33.822577,50.056772,14.961085,30.595194,7.744075,1.921828,14.160990


### Part 2) Hyperparameeters and neural network training

In this part you will be implementing a neural network and will investigate the impact of hyperparameters in model's performance.

1.   Train a neural network with 2 layers (one hidden layer and one output layer).
2.   Search for the most appropriate hyperparameter setting for your model. These hyperparameters include but not limited to (number of epochs, batch size, learning rate, optimizer, momentum).
3. Describe your findings in the text cell below.

In [7]:
## YOUR CODE HERE ##

[YOUR ANSWER HERE]

### Part 3) Understanding neural network architecture

In this part you will experiment with different deep neural network architectures to understand the impact of the architecture on model's accuracy and generalization.

1.   Run an experiment on a 2-layer neural network to investigate the impact of the number of neurons in the hidden layer on model's performance on the training and test data. Plot the result of your experiment.

2. Run the same experiment as the previous part but use depth 3 (2 hidden layers and one output layer). Plot the result of your experiment.

3.   Run an experiment on the impact of the depth of the model on model's performance on the training and test data. For this experiment, use 5 neurons in each hidden layer. Plot the result of your experiment.

4. Describe your findings in the text cell below.




In [8]:
## YOUR CODE HERE ##

[YOUR ANSWER HERE]

### Part 4) Feature selection and final results

Since the number of feeatures will impact model's complexity and that not all features contribute equally to the model's performance, in this part you are asked to perform feature selection, model training, and reporting final results.

Your tasks are:
1.   Based on the experiments you have conducted (and looking into the weights associated to features), perform feature selection.
2. Train an optimum neural network architecture (you will need to do some experiments to find it) with appropriate hyperparameters (you will need to do hyperparameter tuning).
3. Report the Loss of your model on both training and test data. In addition, plot the prediction vs. target labels for both training and test data.


In [9]:
## YOUR CODE HERE ##

[YOUR ANSWER HERE]

### Part 5) State-specific prediction

Thinking about the structure of the data, please provide a description of technique(s) you can use to perform predictions for a given State based on its own data (which reduces the bias in prediction). 

[YOUR ANSWER HERE]

### (BONUS) Part 6) Implement one of your responses to the previous part for an extra credit

To receive an extra credit, you should show the performance boost you observed for the State-specific architecture. 

In [10]:
## YOUR CODE HERE ##